In [44]:
import csv
from datetime import datetime

arcpy.env.workspace = r'C:\School\GEOG 8294\Final Project Data Testing\Final_Scratchspace\Final_Scratchspace.gdb'

def edit_route_times(output_folder, in_csv_name):
    in_table = "Stops_Geo_Projected"
    sql_query = "departure_time < '24:00:00'"
    arcpy.SelectLayerByAttribute_management(in_table, "NEW_SELECTION", sql_query)
    out_table = output_folder + '\stop_times_old.csv'
    arcpy.ExportTable_conversion(in_table, out_table)
    
    changes_data = []
    in_csv = output_folder + in_csv_name
    with open(in_csv, 'r') as changes_file:
        changes_reader = csv.DictReader(changes_file)
        for row in changes_reader:
            changes_data.append(row)
            
    stop_times_data = []
    with open(out_table, 'r') as stop_times_file:
        stop_times_reader = csv.DictReader(stop_times_file)
        for row in stop_times_reader:
            stop_times_data.append(row)

    for change_row in changes_data:
        route_value = change_row['route']
        time_change = datetime.strptime((change_row['time_change']), '%H:%M:%S')

        matched_rows = [row for row in stop_times_data if row['route'] == route_value]

        for row in matched_rows:
            current_arrival_time = datetime.strptime((row['arrival_time']), '%H:%M:%S')
            current_departure_time = datetime.strptime((row['departure_time']), '%H:%M:%S')
            updated_arrival_time = current_arrival_time - time_change
            updated_departure_time = current_departure_time - time_change
            row['arrival_time'] = str(updated_arrival_time)
            row['departure_time'] = str(updated_departure_time)

    output_table = output_folder + '/stop_times.txt'
    with open(output_table, 'w', newline = '') as updated_file:
        fieldnames = stop_times_data[0].keys()
        writer = csv.DictWriter(updated_file, fieldnames = fieldnames)
        writer.writeheader()
        writer.writerows(stop_times_data)
        
edit_route_times(r'C:\School\GEOG 8294\Final Project Data Testing\Data\GTFS_Updated_Scenario_1', '\changes.csv')

In [9]:
import shutil
import os

def copy_files_to_new_folder(input_folder, output_folder):
    input_files = ['agency.txt', 'calendar.txt', 'calendar_dates.txt', 'feed_info.txt', 'linked_datasets.txt', 'routes.txt', 'shapes.txt', 'stops.txt', 'trips.txt', 'vehicles.txt']
    for file in input_files:
        source_path = os.path.join(input_folder, file)
        destination_path = os.path.join(output_folder, file)
        shutil.copyfile(source_path, destination_path)
        
copy_files_to_new_folder(r'C:\School\GEOG 8294\Final Project Data Testing\Data\GTFS', r'C:\School\GEOG 8294\Final Project Data Testing\Data\GTFS_Updated_Scenario_2')